# Model

In [123]:
# For multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [28]:
inputfilename = f'{DATASET_FOLDER}emdat_with_precipitation-2020-12-10.csv.gz'
total_data = pd.read_csv(inputfilename)
total_data.head(3)

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,Entry Criteria,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max
0,1906-0023-BEL,1906,23,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,Louvain region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,5.0,14.0,1906,5.0,14.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300,1906-05-14,13.00,13.0,13.0,0.00,1,478.000000,39.833333,117.0,488.000000,22.181818,117.0,1094.000000,26.047619,117.0
1,1906-0024-BEL,1906,24,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,4.0,28.0,1906,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300,1906-04-28,62.00,62.0,62.0,0.00,1,346.000000,28.833333,62.0,650.000000,29.545455,107.0,650.000000,15.476190,107.0
2,1926-0008-IND,1926,8,Natural,Hydrological,Flood,Riverine flood,NaN,OFDA,India,IND,Southern Asia,Asia,Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1926,7.0,19.0,1926,7.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.940236,1926-07-19,132.19,277585.0,3353.0,259.28,2100,1367.419998,113.951667,4854.0,2351.459991,106.884545,4854.0,4504.939987,107.260476,4854.0


In [29]:
total_data[['Total Deaths', 'No Injured', 'No Affected', 'No Homeless', 'Total Affected']].describe()

,Total Deaths,No Injured,No Affected,No Homeless,Total Affected
count,2.967000e+03,613.000000,2.885000e+03,6.400000e+02,3.300000e+03
mean,7.934816e+02,1951.712887,1.230460e+06,1.211941e+05,1.099587e+06
std,3.673722e+04,16281.917010,9.521997e+06,7.996411e+05,9.115132e+06
min,1.000000e+00,1.000000,2.000000e+01,9.000000e+00,3.000000e+00
25%,6.000000e+00,12.000000,2.000000e+03,1.470000e+03,1.816500e+03
50%,1.700000e+01,33.000000,1.500000e+04,5.500000e+03,1.207750e+04
75%,5.000000e+01,113.000000,1.068460e+05,2.500000e+04,9.938875e+04
max,2.000000e+06,249378.000000,2.230000e+08,1.585000e+07,2.389730e+08


### selecting features

In [30]:
## handling missing data in total affected column :

affected_by_country_per_year=total_data[['Region','ISO','Total Affected','Start Year']].groupby(by=['Region','ISO','Start Year']).mean()
affected_by_region_per_year=total_data[['Region','Total Affected','Start Year']].groupby(by=['Region','Start Year']).mean()
affected_by_region_total=total_data[['Region','Total Affected']].groupby(by=['Region']).mean()
# if we don't have the mean number of total affected in a country for a given year
#we take the mean of the region for the given year, and if we don't have both we take the mean over all the period 
for i in affected_by_region_per_year.index:
    if math.isnan(affected_by_region_per_year.loc[i,'Total Affected']):
        affected_by_region_per_year.loc[i,'Total Affected']=affected_by_region_total.loc[i[0],'Total Affected']
        
for i in affected_by_country_per_year.index:
    if math.isnan(affected_by_country_per_year.loc[i,'Total Affected']):
        affected_by_country_per_year.loc[i,'Total Affected']=affected_by_region_per_year.loc[(i[0],i[2]),'Total Affected']
affected_by_country_per_year

Total Affected
Region                    ISO Start Year                
Australia and New Zealand AUS 1955           4495.075000
                              1971           4495.075000
                              1973          12000.000000
                              1974           4495.075000
                              1975           4495.075000
...                                                  ...
Western Europe            LUX 1993          37866.666667
                          NLD 1953         300000.000000
                              1992          15438.164179
                              1993          13000.000000
                              1998           2000.000000

[2136 rows x 1 columns]

In [31]:
## handling missing data in total affected column :
for i in total_data.index:
    if math.isnan(total_data.loc[i,'Total Affected']):
        total_data.loc[i,'Total Affected']=affected_by_country_per_year.loc[(total_data.loc[i,'Region'],total_data.loc[i,'ISO'],total_data.loc[i,'Start Year']),'Total Affected']

In [34]:
X=total_data[['ISO', 'last_20_days_max']]

y=total_data['Total Affected']

In [35]:
X

,ISO,last_20_days_max
0,BEL,117.0
1,BEL,107.0
2,IND,4854.0
3,DZA,390.0
4,IND,3162.0
...,...,...
3975,PNG,4849.0
3976,POL,688.0
3977,TZA,1400.0
3978,UKR,381.0


In [37]:
## TODO : Load avg_rain to use it
# avg_rain_filename = f'{DATASET_FOLDER}projection_precipitation_for_prediction-2020-12-10.csv.gz'
# df_tmp = pd.read_csv(outfilename)

In [44]:
from sklearn.preprocessing import OneHotEncoder

encoder_training = OneHotEncoder(sparse=False)
encoded_country = encoder_training.fit(np.array(X.ISO).reshape(-1, 1))
#encoded_country.transform(np.array(X.ISO).reshape(-1, 1))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
transformed = encoded_country.transform(X.ISO.to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(transformed, columns=encoded_country.get_feature_names())
#concat with original data
data = pd.concat([X, ohe_df], axis=1).drop(['ISO'], axis=1)
data.head(1)

,last_20_days_max,x0_AFG,x0_AGO,x0_ALB,x0_ARE,x0_ARG,x0_ARM,x0_ASM,x0_AUS,x0_AUT,x0_AZE,x0_BDI,x0_BEL,x0_BEN,x0_BFA,x0_BGD,x0_BGR,x0_BHS,x0_BIH,x0_BLR,x0_BLZ,x0_BOL,x0_BRA,x0_BRB,x0_BWA,x0_CAF,x0_CAN,x0_CHE,x0_CHL,x0_CHN,x0_CIV,x0_CMR,x0_COD,x0_COG,x0_COL,x0_CRI,x0_CUB,x0_CZE,x0_DEU,x0_DOM,x0_DZA,x0_ECU,x0_EGY,x0_ESP,x0_ETH,x0_FIN,x0_FJI,x0_FRA,x0_FSM,x0_GAB,x0_GBR,x0_GEO,x0_GHA,x0_GIN,x0_GLP,x0_GMB,x0_GRC,x0_GTM,x0_GUF,x0_GUY,x0_HND,x0_HRV,x0_HUN,x0_IDN,x0_IND,x0_IRL,x0_IRN,x0_IRQ,x0_ISL,x0_ISR,x0_ITA,x0_JAM,x0_JOR,x0_JPN,x0_KAZ,x0_KEN,x0_KGZ,x0_KIR,x0_KOR,x0_KWT,x0_LAO,x0_LBN,x0_LBR,x0_LBY,x0_LKA,x0_LSO,x0_LTU,x0_LUX,x0_MAR,x0_MDA,x0_MDG,x0_MDV,x0_MEX,x0_MHL,x0_MKD,x0_MLI,x0_MMR,x0_MNE,x0_MNG,x0_MOZ,x0_MRT,x0_MUS,x0_MWI,x0_MYS,x0_NAM,x0_NER,x0_NGA,x0_NIC,x0_NLD,x0_NOR,x0_NPL,x0_NZL,x0_OMN,x0_PAK,x0_PAN,x0_PER,x0_PHL,x0_PNG,x0_POL,x0_PRI,x0_PRK,x0_PRT,x0_PRY,x0_PYF,x0_ROU,x0_RUS,x0_RWA,x0_SAU,x0_SDN,x0_SEN,x0_SLB,x0_SLV,x0_SRB,x0_SUR,x0_SVK,x0_SVN,x0_SWE,x0_SWZ,x0_SYC,x0_SYR,x0_TCD,x0_TGO,x0_THA,x0_TJK,x0_TKM,x0_TTO,x0_TUN,x0_TUR,x0_TZA,x0_UKR,x0_URY,x0_USA,x0_UZB,x0_VEN,x0_VNM,x0_VUT,x0_ZAF,x0_ZMB,x0_ZWE
0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,107.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4854.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,390.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3162.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:

# pipeline1 = Pipeline([
#                 ('OneHotEncoder', encoder_training())
#             ])
# pipeline1.fit(trainingdata.column1.values.reshape(-1,1))
# joblib.dump(pipeline1,"pipeline1.joblib")
# # Production environment
# pipeline1 = joblib.load('pipeline1.joblib')
# momo = pipeline1.transform(productiondata.column1.values.reshape(-1,1)).toarray()

In [48]:
X=data

In [51]:
#X_train=X[X['Start Year']<=2015]
#y_train=y[X['Start Year']<=2015]
#print('X_train ' , X_train.shape)
#print('y_train ' , X_train.shape)

#X_test=X[X['Start Year']>2015]
#y_test=y[X['Start Year']>2015]

#print('X_test', X_test.shape)
#print('y_test', X_test.shape)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
print('X_train ' , X_train.shape)
print('y_train ' , X_train.shape)
print('X_test', X_test.shape)
print('y_test', X_test.shape)

X_train  (3184, 159)
y_train  (3184, 159)
X_test (796, 159)
y_test (796, 159)


In [53]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,max_error
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import max_error, explained_variance_score

In [63]:
regr = GradientBoostingRegressor()
regr.fit(X_train, y_train)
y_pred=regr.predict(X_test)

In [64]:
# he coefficient of determination R^2 of the prediction
print('The best possible score is 1.0')
regr.score(X_test,y_test)

The best possible score is 1.0


-0.3169217908926687

In [65]:
print('The best possible score is 1.0')

explained_variance_score(y_test, y_pred)

The best possible score is 1.0


-0.3154138312614079

# Predicting flood events :

In [204]:
DATASET_FOLDER = '../../datasets/'
proj_filename = f'{DATASET_FOLDER}projection_precipitation_for_prediction-2020-12-10.csv.gz'
df_proj = pd.read_csv(proj_filename)
df_proj.head()

,year,ISO3,projection_rcp,rain_anomaly,avg_hist_rain,est_max_rain
0,2031,JOR,rcp26,-9.74730,9.007923,-0.739377
1,2081,JOR,rcp26,-10.25700,9.007923,-1.249077
2,2059,JOR,rcp60,-3.76900,9.007923,5.238923
3,2072,JOR,rcp26,0.57577,9.007923,9.583693
4,2096,JOR,rcp45,-3.55980,9.007923,5.448123


In [205]:
# Print missing countries in EMDAT
df_proj[~ df_proj.ISO3.isin(encoded_country.categories_[0]) ].ISO3.unique()

array(['MLT', 'YEM', 'EST', 'VCT', 'HTI', 'QAT', 'SLE', 'AND', 'COM',
       'DMA', 'TUV', 'LVA', 'BHR', 'TLS', 'LCA', 'PLW', 'BTN', 'ERI',
       'TON', 'ATG', 'GNB', 'STP', 'GRL', 'BRN', 'CYP', 'UGA', 'LIE',
       'FRO', 'CPV', 'SOM', 'GRD', 'NCL', 'DNK', 'MCO', 'DJI', 'KNA',
       'SSD', 'WSM', 'KHM', 'SGP'], dtype=object)

In [206]:
# Remove countries we do not know in EMDAT
len(df_proj)
df_proj = df_proj[ df_proj.ISO3.isin(encoded_country.categories_[0]) ].reset_index()
len(df_proj)

9202

7328

In [207]:
df_proj = df_proj.drop('index', axis=1)

In [208]:
df_proj.projection_rcp.unique()
rcp = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

array(['rcp26', 'rcp60', 'rcp45', 'rcp85'], dtype=object)

In [209]:
#df_proj85 = df_proj.query("projection_rcp == 'rcp85'")

In [212]:
df_proj_X = df_proj[['ISO3', 'est_max_rain']]
#df_proj_X[df_proj_X.est_max_rain < 0] = 0
len(df_proj_X)

7328

In [213]:
df_proj_X.head(2)

,ISO3,est_max_rain
0,JOR,-0.739377
1,JOR,-1.249077


In [214]:
df_proj_X[df_proj_X.isna().any(axis=1)]

,ISO3,est_max_rain


## Apply transform

In [295]:
transformed = encoded_country.transform(df_proj_X.ISO3.to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(transformed, columns=encoded_country.get_feature_names())
#concat with original data
data = pd.concat([df_proj_X, ohe_df], axis=1).drop(['ISO3'], axis=1)
data.head(10)

,est_max_rain,x0_AFG,x0_AGO,x0_ALB,x0_ARE,x0_ARG,x0_ARM,x0_ASM,x0_AUS,x0_AUT,x0_AZE,x0_BDI,x0_BEL,x0_BEN,x0_BFA,x0_BGD,x0_BGR,x0_BHS,x0_BIH,x0_BLR,x0_BLZ,x0_BOL,x0_BRA,x0_BRB,x0_BWA,x0_CAF,x0_CAN,x0_CHE,x0_CHL,x0_CHN,x0_CIV,x0_CMR,x0_COD,x0_COG,x0_COL,x0_CRI,x0_CUB,x0_CZE,x0_DEU,x0_DOM,x0_DZA,x0_ECU,x0_EGY,x0_ESP,x0_ETH,x0_FIN,x0_FJI,x0_FRA,x0_FSM,x0_GAB,x0_GBR,x0_GEO,x0_GHA,x0_GIN,x0_GLP,x0_GMB,x0_GRC,x0_GTM,x0_GUF,x0_GUY,x0_HND,x0_HRV,x0_HUN,x0_IDN,x0_IND,x0_IRL,x0_IRN,x0_IRQ,x0_ISL,x0_ISR,x0_ITA,x0_JAM,x0_JOR,x0_JPN,x0_KAZ,x0_KEN,x0_KGZ,x0_KIR,x0_KOR,x0_KWT,x0_LAO,x0_LBN,x0_LBR,x0_LBY,x0_LKA,x0_LSO,x0_LTU,x0_LUX,x0_MAR,x0_MDA,x0_MDG,x0_MDV,x0_MEX,x0_MHL,x0_MKD,x0_MLI,x0_MMR,x0_MNE,x0_MNG,x0_MOZ,x0_MRT,x0_MUS,x0_MWI,x0_MYS,x0_NAM,x0_NER,x0_NGA,x0_NIC,x0_NLD,x0_NOR,x0_NPL,x0_NZL,x0_OMN,x0_PAK,x0_PAN,x0_PER,x0_PHL,x0_PNG,x0_POL,x0_PRI,x0_PRK,x0_PRT,x0_PRY,x0_PYF,x0_ROU,x0_RUS,x0_RWA,x0_SAU,x0_SDN,x0_SEN,x0_SLB,x0_SLV,x0_SRB,x0_SUR,x0_SVK,x0_SVN,x0_SWE,x0_SWZ,x0_SYC,x0_SYR,x0_TCD,x0_TGO,x0_THA,x0_TJK,x0_TKM,x0_TTO,x0_TUN,x0_TUR,x0_TZA,x0_UKR,x0_URY,x0_USA,x0_UZB,x0_VEN,x0_VNM,x0_VUT,x0_ZAF,x0_ZMB,x0_ZWE
0,-0.739377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.249077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.238923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9.583693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.448123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0

In [296]:
len(data)

7328

In [297]:
proj_affected=regr.predict(data)

In [298]:
ohe_df = pd.DataFrame(proj_affected, columns=['affected'])
ohe_df

,affected
0,141408.414439
1,141408.414439
2,141408.414439
3,141408.414439
4,141408.414439
...,...
7323,141408.414439
7324,141408.414439
7325,141408.414439
7326,141408.414439


In [299]:
final_projection = pd.concat([df_proj, ohe_df], axis=1)
final_projection.head(2)

,year,ISO3,projection_rcp,rain_anomaly,avg_hist_rain,est_max_rain,affected
0,2031,JOR,rcp26,-9.7473,9.007923,-0.739377,141408.414439
1,2081,JOR,rcp26,-10.2570,9.007923,-1.249077,141408.414439


In [300]:
### Add decade
def decade(year):
    return int(str(year)[:3]+'0')
final_projection['Decade'] = final_projection.year.apply(decade)
# Add Disaster_Type
final_projection['Disaster_Type'] = 'Flood'
final_projection['Financial_Impact'] = final_projection['affected'] * 1_233
final_projection.head(2)

,year,ISO3,projection_rcp,rain_anomaly,avg_hist_rain,est_max_rain,affected,Decade,Disaster_Type,Financial_Impact
0,2031,JOR,rcp26,-9.7473,9.007923,-0.739377,141408.414439,2030,Flood,1.743566e+08
1,2081,JOR,rcp26,-10.2570,9.007923,-1.249077,141408.414439,2080,Flood,1.743566e+08


In [301]:
final_projection.columns

Index(['year', 'ISO3', 'projection_rcp', 'rain_anomaly', 'avg_hist_rain',
       'est_max_rain', 'affected', 'Decade', 'Disaster_Type',
       'Financial_Impact'],
      dtype='object')

In [302]:
# Add UN_Geosheme_Subregion

In [303]:
total_data.columns

Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria',
       'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin',
       'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal',
       'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI', 'date', ' avg_rain', 'sum_rain',
       'max_rain', 'stddev_rain', 'station_count', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum',
       'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum',
       'last_20_days_avg', 'last_20_days_max', 'flood'],
      dtype='

In [304]:
region = total_data[['ISO', 'Region']].drop_duplicates().dropna().reset_index(drop=True)
final_projection = final_projection.merge(region, left_on='ISO3', right_on='ISO', how='inner')

In [305]:
final_projection.columns

Index(['year', 'ISO3', 'projection_rcp', 'rain_anomaly', 'avg_hist_rain',
       'est_max_rain', 'affected', 'Decade', 'Disaster_Type',
       'Financial_Impact', 'ISO', 'Region'],
      dtype='object')

In [308]:
# Group by Decade, UN_Geosheme_Subregion
df_temp = final_projection[['projection_rcp', 'rain_anomaly', 'affected', 'Decade', 'Disaster_Type',
       'Financial_Impact', 'Region']]
final_projection_by_decade = df_temp.groupby(['Decade', 'Region', 'projection_rcp', 'Disaster_Type']).agg(Financial_Impact=('Financial_Impact', 'sum'),
    Rain=('rain_anomaly', 'mean'), Human_Impact=('affected', 'sum'), DO=('rain_anomaly', 'count')).reset_index()


In [309]:
final_projection_by_decade = final_projection_by_decade.rename(columns={'projection_rcp':'RCP', 'Region':'UN_Geosheme_Subregion'})

In [310]:
# Decade 	UN_Geosheme_Subregion 	Disaster_Type 	RCP 	Financial_Impact 	Human_Impact 	DO 	nb_days_with_rain_>_50mm

In [311]:
final_projection_by_decade.RCP.replace({'rcp26': 2.6}, inplace=True)
final_projection_by_decade.RCP.replace({'rcp45': 4.5}, inplace=True)
final_projection_by_decade.RCP.replace({'rcp60': 6.0}, inplace=True)
final_projection_by_decade.RCP.replace({'rcp85': 8.5}, inplace=True)

In [312]:
final_projection_by_decade.astype({
    'Financial_Impact': 'int64',
    'Human_Impact': 'int64',
    'Rain': 'int64'
})

,Decade,UN_Geosheme_Subregion,RCP,Disaster_Type,Financial_Impact,Rain,Human_Impact,DO
0,2020,Australia and New Zealand,2.6,Flood,697426300,6,565633,4
1,2020,Australia and New Zealand,4.5,Flood,697426300,2,565633,4
2,2020,Australia and New Zealand,6.0,Flood,348713150,7,282816,2
3,2020,Australia and New Zealand,8.5,Flood,348713150,2,282816,2
4,2020,Caribbean,2.6,Flood,1917922325,-6,1555492,11
...,...,...,...,...,...,...,...,...
667,2090,Western Asia,8.5,Flood,3138418350,2,2545351,18
668,2090,Western Europe,2.6,Flood,1569209175,8,1272675,9
669,2090,Western Europe,4.5,Flood,1743565750,3,1414084,10
670,2090,Western Europe,6.0,Flood,1917922325,9,1555492,11


In [314]:
outfilename = f'{DATASET_FOLDER}projection_precipitation_with_prediction-2020-12-12.csv.gz'
final_projection_by_decade.to_csv(outfilename, index=False, compression='gzip')

In [315]:
df_tmp = pd.read_csv(outfilename)
df_tmp.head(3)

,Decade,UN_Geosheme_Subregion,RCP,Disaster_Type,Financial_Impact,Rain,Human_Impact,DO
0,2020,Australia and New Zealand,2.6,Flood,6.974263e+08,6.672918,565633.657756,4
1,2020,Australia and New Zealand,4.5,Flood,6.974263e+08,2.440525,565633.657756,4
2,2020,Australia and New Zealand,6.0,Flood,3.487132e+08,7.790505,282816.828878,2


In [316]:
df_tmp.RCP.unique()

array([2.6, 4.5, 6. , 8.5])